In [21]:
# docker run -p 8888:8888 jupyter/datascience-notebook:latest
import os
import datetime
import numpy as np
import pandas as pd

import qgrid
qgrid.enable()
# qgrid.disable()

import api
import bars

%load_ext autoreload
%autoreload 2

# import ipdb; ipdb.set_trace(context=10)
# import pudb; pu.db

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
symbol='GLD'
fmat = 'parquet'
date = '2019-05-09'

ts = api.read_matching_files(f"../data/ticks/{fmat}/{symbol}/{symbol}_{date}.{fmat}", format=fmat)
# ts = bars.epoch_to_dt(ts)

In [26]:
bars_5m = bars.time_bars(ts, freq='5min')

IndexError: list index out of range